In [3]:
'''
ssg_ep의 경우 10G 다운받는과정만 6시간정도 걸리는데, 
대부분 mysql insert과정에서 시간소요되는듯
다운을 먼저 받고 batch insert를 날리던가 
db에 저장하지 않는 방식으로 진행하면 속도문제 해결

181서버에서 10G다운받는 시간 약2분
pandas를 통해 처리후 파일생성(zip) 시간 약3분

증분업데이트도 이런식으로 처리하면 서버부하 거의없지않을까?
피드내용 확인하는 로직만 필요없다면 가능할것같음

fastapi로 api서버 구동해서 스케줄마다 실행시키면 관리하기 수월할듯?

// 기본
class FeedConvert(config):
    def load
        
    def convert
        df.apply(lambda x : customConvert(x), axis=1)    
        df.apply(lambda x : mediaConvert(x), axis=1)
        
    def write
        
    def upload
        

// 고객사별 조건
def customConvert(df)    
    -lambda로 실행
    -관리편의를 위해 로직을 catalog id기준으로 묶어놓음
    -add/remove
    -fit field
    -replace value
    -set link
    -default value

// 매체별 조건 (생략?)
def FacebookConvert()
def GoogleConvert()
def CriteoConvert()

// test 용이하도록

// log 확인 

// linux process 관리가능? 

'''

'\n// 기본\nclass FeedConvert(config):\n    def load\n        \n    def convert\n        df.apply(lambda x : customConvert(x), axis=1)    \n        df.apply(lambda x : mediaConvert(x), axis=1)\n        \n    def write\n        \n    def upload\n        \n\n// 고객사별 조건\ndef customConvert(df)    \n    -lambda로 실행\n    -관리편의를 위해 로직을 catalog id기준으로 묶어놓음\n    -add/remove\n    -fit field\n    -replace value\n    -set link\n    -default value\n\n// 매체별 조건 (생략?)\ndef FacebookConvert()\ndef GoogleConvert()\ndef CriteoConvert()\n\n// test 용이하도록\n\n// log 확인 \n\n// linux process 관리가능? \n\n'

### import

In [1]:
import pandas as pd
from lxml import etree
import os
import gc
import zipfile

### parameter

In [33]:
config = {
          'info': 
          {
            'media': 'facebook',
            'catalog_id' : '268046537186348',
            'feed_id' : '',
            'name': 'ssg_ep_test',
          },
          'read':
          {
            'path' : './data/dumy_10g.csv',
            'format' : 'csv',
            'encoding' : 'utf-16',
            'chunkSize' : 100000
          },
          'write':{
            'path': './data/saved_test.txt',
            'zipPath': './data/zip_test.zip',
            'encoding' : 'utf-8'
          },
          'field' : 
            {
              'id' : 'id', 
              'title' : 'title',
              'link' : 'link',
              'image_link' : 'image_link',
              'price' : 'value',
              'brand' : 'brand',
              'description' : 'description',
              'availability' : 'availability'
            },
          'filter': # ep to feed
            { 
              #'include' : {'id':['','','','']}, # 특정 값 포함           
              'exclude' : {'title': ['마스크','소독제','손소독겔','새니타이저','손살균제','sanitizers','disinfection wipes']}, #키워드 제외
              'replace' : {'title': {'before':'after'}}, # filed의 before->after 치환
              'custom' : {'title': }
            }
         }

### filter class

In [142]:
class FeedConvert():
    def __init__(self,config):
        self.config = config
        self.result = pd.DataFrame()

    def init(self, dataframe):
        # field
        keys = list(self.config['field'].keys())
        self.result = dataframe[keys] # 필요컬럼만 추출
        self.result.rename(columns= self.config['field'], inplace=True) # key 수정        

    def commonFilter(self):
        # include
        if 'include' in self.config['filter']:
            keys = self.config['filter']['include'].keys()
            for key in keys :
                value = self.config['filter']['include'][key]
                self.result = self.result.loc[self.result[key].isin(value)] #포함
        # exclude
        if 'exclude' in self.config['filter']:
            keys = self.config['filter']['exclude'].keys()
            for key in keys :
                value = self.config['filter']['exclude'][key]
                self.result = self.result.loc[~self.result[key].isin(value)] #제외

        # replace
        if 'replace' in self.config['filter']:
            keys = self.config['filter']['replace'].keys()
            for key in keys : 
                value = self.config['filter']['replace'][key]
                before = list(value.keys())[0]
                after = list(value.values())[0]                
                self.result[key] = self.result[key].str.replace(before, after, regex=True) # regex사용 replace        

    # 특수한 개별로직인 경우 config와 매칭시켜 개별관리
    def customFilter(self):
        media = self.config['info']['media']
        catalog_id = self.config['info']['catalog_id']
        name = self.config['info']['name']

        if media == 'facebook' :
            if name == 'ssg_ep_test' and catalog_id == '268046537186348' :
                self.result['link'] = self.result['link'] + "&test=1"

            elif name == 'test' and catalog_id == '0000000000000':
                pass
        

    # 매체별 기본값 등    
    def mediaFilter(self):
        if self.config['info']['media'] == 'facebook' :
            # availavility
            if 'availavility' not in self.result :
                self.result['availability'] = 'in stock'
            # condition
            if 'condition' not in self.result : 
                self.result['condition'] = 'new'
            # description
            if 'description' not in self.result :
                self.result['description'] = self.result['title']


    # 실행함수
    def run(self, dataframe):
        # 최초 초기화 set Fields
        self.init(dataframe)
        
        # 공통 filter
        self.commonFilter()
        
        # 개별 filter
        self.customFilter()
        
        # 매체별 공통 filter
        self.mediaFilter()
        
        return self.result

In [146]:

class FeedConvertProcess():        
    def __init__(self, config):
        self.config = config
        self.feedConvert = FeedConvert(self.config) #convert관리 class
        
        
    def epLoad(self):        
        # pixel로 관리되는 tracking data
        
        # ep데이터 있는경우
        if config['read']['format'] == 'xml':
            # xml은 따로 로드함
            result = etree.iterparse(self.config['read']['path'])
            
        else:
            # chunksize 단위로 쪼개서 로드        
            result = pd.read_csv(self.config['read']['path'],
                                 chunksize=self.config['read']['chunkSize'],
                                 delimiter=',',
                                 encoding=self.config['read']['encoding'])

        # iterator포함 객체로 출력
        return result
        
        
    # def ep2Feed(self, df):        
    #     # custom convert    
    #     # media convert    
    #     return result

    def feedWrite(self, num, df):
        if num == 0:
            df.to_csv(self.config['write']['path'], index=True, sep='\t', mode='w', encoding=self.config['write']['encoding'])
        else:
            df.to_csv(self.config['write']['path'], index=True, sep='\t', mode='a', encoding=self.config['write']['encoding'], header=False)
        
        print((num+1)*self.config['read']['chunkSize'], end='..', flush=True)    
    
    def feedUpload(self):
        # zip
        zf = zipfile.ZipFile(self.config['write']['zipPath'], mode='w')
        zf.write(self.config['write']['path'], compress_type=zipfile.ZIP_DEFLATED)
        zf.close()
        # upload
        
        # delete


    def execute(self):                
        for num, chunkDF in enumerate(self.epLoad()): # chunk load
            # convert
            chunkDF = self.feedConvert.run(chunkDF)

            # file write
            self.feedWrite(num, chunkDF)
            
            # gc
            del [[chunkDF]]
            gc.collect()

            # break
            # if num >= 0:
            #     break
                
        
        self.feedUpload()
        

In [147]:
fcp = FeedConvertProcess(config)

In [148]:
%%time
fcp.execute()

100000..200000..300000..400000..500000..600000..700000..800000..900000..1000000..1100000..1200000..1300000..1400000..1500000..1600000..1700000..1800000..1900000..2000000..2100000..2200000..2300000..2400000..2500000..2600000..2700000..2800000..2900000..3000000..3100000..3200000..3300000..3400000..3500000..3600000..3700000..3800000..3900000..4000000..4100000..4200000..4300000..4400000..4500000..4600000..4700000..4800000..4900000..5000000..5100000..5200000..5300000..5400000..5500000..5600000..5700000..5800000..5900000..6000000..6100000..6200000..6300000..6400000..6500000..6600000..6700000..6800000..6900000..7000000..7100000..7200000..7300000..7400000..7500000..7600000..7700000..7800000..7900000..8000000..8100000..8200000..8300000..8400000..8500000..8600000..8700000..8800000..8900000..9000000..9100000..9200000..9300000..9400000..9500000..9600000..Wall time: 4min 3s


In [ ]:
import requests

# facebook api
class FacebookApiManager() :
    def __init__(self):
        pass

    # app인증 후 토큰발급
    def getAccessToken(self) :
        version = '5.0'
        client_id = ''
        redirect_uri = ''
        client_secret = ''
        authorization_code = ''
        url = '''https://graph.facebook.com/v{version}/oauth/access_token?
                client_id={client_id}&redirect_uri={redirect_uri}&client_secret={client_secret}&code={authorization_code}'''
                .format(version='5.0',client_id=client_id, redirect_id=redirect_id, authorization_code=authorization_code)

        # request                 
        pass

    # 장기실행 엑세스 토큰 
    def getLongAccessToken(self) :        
        version = '5.0'
        client_id = ''
        redirect_uri = ''
        client_secret = ''
        access_token = ''
        url = '''https://graph.facebook.com/v{version}/oauth/access_token?
                grant_type=fb_exchange_token&client_id={client_id}&redirect_uri={redirect_uri}&client_secret={client_secret}&fb_exchange_token={access_token}'''
                .format(version='5.0',client_id=client_id, redirect_id=redirect_id, access_token=access_token)

        # request                 
        pass

    def feedUpload(self) :

        feed_id = ''
        version = '5.0'
        access_token = 'EAACpvLy2z30BAAwB8uF80ZCIk4q5W1J71qdpO7FHS2CkYLZCVR9GGHuzre2NINYddP4ynDNm7KTFpZBMCQjN1lPyX0DugC8O2yKKwPt75Msq4dnrhkWCqVUHHFgTnhdU2fhhVg7dgJ7DLrsgpZB2iEBPZAbpBXaYIgmcor5JGRj5eQmzv7C8x'
        feed_url = 'http://api.dmcf1.com/feed/482060879406329/watermark_se_482060879406329.json.gz'
        url = 'https://graph.facebook.com/v{version}/{feed_id}/uploads?access_token={access_token}&url={url}'.format(version='5.0', feed_id='', access_token=token, url=feed_url)

        # request
        response = requests.post(url)
        print(response.status_code)
        print(response.text)
        pass

In [3]:
ep_path='C:\\Users\\shsun\\Documents\\workspace\\project\\p1\\f1_feed_change_min\\fastapi\\data\\ep\\ep_catalogid_154972755345007.tsv'

In [13]:
load = pd.read_csv(ep_path,
                    chunksize=1,
                    nrows=1,
                    header=0,
                    sep='\t',
                    index_col=None,
                    lineterminator='\n',
                    error_bad_lines=False,                  
                    encoding='utf-8')

In [14]:
for chunkDF in enumerate(load):    
    print(chunkDF)
    break

(0,       id               title  price_pc  \
0  21379  [강성은명과] 동백 세트 639g     39000   

                                                link  \
0  https://www.hellonature.co.kr/fdp001.do?goTo=d...   

                                          image_link category_name1  \
0  https://img-ex.hellonature.co.kr/images/ex/PGD...        생활·선물세트   

  category_name2 category_name3 category_name4  brand  \
0           선물세트            기획전        음료/차/다과  강성은명과   

                     maker   origin  coupon  point  review_count  shipping  
0  생산자 : 강성은명과 / 서울시 서대문구   상세페이지참조     NaN    NaN            13      2500  )
